In [25]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)
# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

In [26]:
# all modules necessary for this nb
import os
import sys
import pickle

import numpy as np
import pylab as pl
from sklearn.covariance import EmpiricalCovariance
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.metrics import silhouette_score as clust_score
from sklearn.preprocessing import StandardScaler
from scipy import stats as sstats

# setting parameters for default matplotlib plots
%matplotlib inline

In [27]:
pl.rcParams['savefig.dpi'] = 300 # dpi for most publications
pl.rcParams['figure.dpi'] = 300 # dpi for most publications
pl.rcParams['xtick.labelsize'] = 7
pl.rcParams['ytick.labelsize'] = 7
pl.rcParams['axes.labelsize'] = 7
from ipywidgets import interact

# needs to find the library of functions
sys.path.append('../../../../../code/')  # to be replaced!

import utils as ut
import plots as pt

In [28]:
# a double percentage sign indicates a magic function. in this case, now we are writing this cell in javascript.

In [29]:
NOTEBOOK_NAME = 'preprocessing'

In [30]:
from pickleshare import PickleShareDB

autorestore_folder = os.path.join(os.getcwd(), 'autorestore', NOTEBOOK_NAME)
db = PickleShareDB(autorestore_folder)
import sys
from workspace import *
import IPython
ip = IPython.get_ipython()

# this will restore all the saved variables. ignore the errors listed.
#load_workspace(ip, db)

# use `save_worspace(db)` to save variables at the end

In [31]:
data_folder = '../data'

In [32]:
traces = np.loadtxt(os.path.join(data_folder, 'C.txt')).T #denoised traces
traces_raw = np.loadtxt(os.path.join(data_folder, 'C_raw.txt')).T
try:
    areas = ut.load_spatial_footprints_A(os.path.join(data_folder, 'A.txt'))
except:
    print 'not 512 by 512'
events = np.loadtxt(os.path.join(data_folder, 'S.txt')).T
dff = np.loadtxt(os.path.join(data_folder, 'C_df.txt')).T
#mean_image, contours = ut.load_spatial_footprints(os.path.join(data_folder, 'Coor.mat'),
#                                                   os.path.join(data_folder, 'Cnn.txt'),
#    

#filename = os.path.join(data_folder, 'behavior.txt')

# adapting above code so we don't have to rename every arudino file to 'behavior.txt' when importing
for file in os.listdir(data_folder):
    if file.endswith("codes.txt"):
        filename = os.path.join(data_folder, file)
    elif 'behavior' in file:
        filename = os.path.join(data_folder, file)
print filename

behavior = ut.read_behavior(filename)
events_list = np.unique([b[1] for b in behavior])

../data/104_071522_D5_codes.txt


In [33]:
# grab time axis from the xml file

import xml.etree.ElementTree as ET
#xmlfile = os.path.join(data_folder, 'tseries.xml')

#adapting above code so we don't have to rename imported .xml file
for file in os.listdir(data_folder):
    if file.endswith(".xml"):
        xmlfile = data_folder + '/' + file
print "I infer the time axis from:\n", xmlfile
tree = ET.parse(xmlfile)
root = tree.getroot()

# unfortunately we miss the first frame
time_ax = np.r_[[child.attrib['absoluteTime']
                 for child in root.iter('Frame')]].astype(float)

I infer the time axis from:
../data/TSeries-07152022-104-365um-3z-000.xml


In [34]:
time_ax

array([   16.388     ,    16.42107047,    16.45414094, ...,  3516.55602438,
        3516.58910152,  3516.62217866])

In [35]:
# sync times
start_2p = ut.parse_behavior(behavior, 'BEGIN')[0]
behavior = [[float(b[0])-start_2p, b[1]] for b in behavior] #speed_behavior must come before this, since that references behavior 
time_ax -= time_ax[0]

In [36]:
# make sure presentations are correct in timing.
print behavior[:5]

[[-6.2970000000000006, 'Speed_0'], [-6.0970000000000013, 'Speed_0'], [-5.897000000000002, 'Speed_0'], [-5.6969999999999992, 'Speed_0'], [-5.4969999999999999, 'Speed_0']]


In [37]:
print len(traces)
print len(time_ax)

7050
56400


In [38]:
ratio = int(np.floor(time_ax.shape[0]/traces.shape[0]))
print ratio

8


In [39]:
time_ax = time_ax[::ratio] # use this if video was averaged and need to adjust xml output to match

In [40]:
print len(traces)
print len(time_ax)

7050
7050


In [41]:
time_ax = time_ax[0:len(traces)] # use this if any video frames were truncated (often need to do this a video is averaged)

In [42]:
print len(traces)
print len(time_ax)

7050
7050


In [43]:
#to correct for different clocking speeds between arduino and 2p software
behavior = ut.sync_behavior_to_xml(time_ax, behavior)

[ 0.9899166   0.98996353  0.98999544  0.97294929  0.99003741  0.99004043
  0.99005106  0.97298059  0.9901211   0.99000261  0.99007333  0.97300869
  0.99008033  0.99007752  0.9900789   0.97298771  0.99000707  0.99008075
  0.99008578  0.97301528  0.99008958  0.9900739   0.99008171  0.97301663
  0.99008922  0.99008907  0.99007803  0.97301254  0.99008602  0.99001251
  0.99007641  0.97299398  0.99006707  0.99007752  0.99008931  0.9730259
  0.99010167  0.9900759   0.99009024  0.97302851  0.99010114  0.99008835
  0.99010033  0.97302369  0.99010081  0.99008371  0.99002907  0.97303178
  0.99010673  0.99010706  0.99007674  0.97302178  0.99010395  0.99008949
  0.99009742  0.97303269  0.99011191  0.99012068  0.99012173  0.97299402
  0.99012924  0.99012559  0.99012805  0.97299887  0.99007468  0.99014149
  0.99007127  0.97305746  0.99006235  0.99012622  0.99009401  0.97303119
  0.990112    0.98998837  0.99002886  0.97303566  0.99011308  0.99011844
  0.99012305  0.97304499  0.99009742  0.99008653  0.

In [44]:
#extract speed value as integer from the string
speed_behavior = [[float(b[0])-float(behavior[0][0]), int(b[1][6:])] for b in behavior if b[1][6:] !=''] # this only works if non-speed event codes are less than 7 characters 

In [45]:
speed_times = np.r_[ut.parse_behavior(behavior, 'Speed')]
speed = []
for x in speed_behavior:
    if x[-1] > 1000:
        speed.append(0)
    else:
        speed.append(x[-1])

In [46]:
# -----------------------------------------------------------
# these times are relative to the single cycle
# and centered around tone onset
#CONTINUOUS = True
CYCLE_START = -3  # seconds
CS_DURATION = 2  # seconds  // IS THIS FIXED?
DELAY = 2
AFTER_DELAY_DURATION = 7.5
US_START = 4
CYCLE_DURATION = abs(CYCLE_START) + CS_DURATION + DELAY + AFTER_DELAY_DURATION

# -----------------------------------------------------------
# these times are absolute times, taken from the arduino file
rewards = np.r_[ut.parse_behavior(behavior, 'REWARD')]
#For times when sucrose reward is actually collected, see 'consumptions' variable created a few cell down
shocks = np.r_[ut.parse_behavior(behavior, 'SHOCK')]
licks = np.r_[ut.parse_behavior(behavior, 'LICK')]
punished = np.r_[ut.parse_behavior(behavior, 'PUNISH')]
avoided = np.r_[ut.parse_behavior(behavior, 'AVOID')]
enabled = np.r_[ut.parse_behavior(behavior, 'ENABLE')]
armed = np.r_[ut.parse_behavior(behavior, 'ARMED')]
CSm_enabled = np.r_[ut.parse_behavior(behavior, 'CSmEN')]
STIM5_ons = ut.parse_behavior(behavior, 'STIM5')
STIM6_ons = ut.parse_behavior(behavior, 'STIM6')
STIM7_ons = ut.parse_behavior(behavior, 'STIM7')
cycles_starts = ut.parse_behavior(behavior, '^STIM_*', offset=CYCLE_START) #looks for arduino line that begins w/ 'TONE'
cycles_ends = ut.parse_behavior(behavior, '^STIM_*', offset=CYCLE_DURATION+CYCLE_START)
cycles = np.r_[zip(cycles_starts,cycles_ends)]
STIMs_ons = ut.parse_behavior(behavior, '^STIM_*')

# -----------------------------------------------------------
# when the experiment starts and ends, in absolute time
# begin_end = ut.parse_behavior(behavior, '[be]')
# when each cycle starts and ends
cycles_starts = ut.parse_behavior(behavior, ('STIM*'), offset=CYCLE_START) #looks for arduino line that begins w/ either O, R, S, or b
cycles_ends = ut.parse_behavior(behavior, ('STIM*'), offset=CYCLE_DURATION+CYCLE_START)
cycles = np.r_[zip(cycles_starts,  # offset will be ADDED, with sign
                cycles_ends)]

# -----------------------------------------------------------
# which trials are of a certain US
is_STIM5t = [any(map(lambda t: (t>=s) and (t<e), STIM5_ons)) for s, e in zip(cycles_starts, cycles_ends)]
is_STIM6t = [any(map(lambda t: (t>=s) and (t<e), STIM6_ons)) for s, e in zip(cycles_starts, cycles_ends)]
is_STIM7t = [any(map(lambda t: (t>=s) and (t<e), STIM7_ons)) for s, e in zip(cycles_starts, cycles_ends)]
is_rewardedt = [any(map(lambda t: (t>=s) and (t<e), rewards)) for s, e in zip(cycles_starts, cycles_ends)]
is_shockedt = [any(map(lambda t: (t>=s) and (t<e), shocks)) for s, e in zip(cycles_starts, cycles_ends)]
is_avoidedt = [any(map(lambda t: (t>=s) and (t<e), avoided)) for s, e in zip(cycles_starts, cycles_ends)]
is_CSm_enabledt = [any(map(lambda t: (t>=s) and (t<e), CSm_enabled)) for s, e in zip(cycles_starts, cycles_ends)]


In [48]:
licks_bs = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START-DELAY,
                                      end=-CYCLE_START)  # w.r.t. cycle start
licks_cs = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START,
                                      end=-CYCLE_START+CS_DURATION)
licks_tc = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START+CS_DURATION,
                                         end=-CYCLE_START+CS_DURATION+DELAY)
licks_cs_tc = 1.*ut.compute_licks_during(licks, cycles,
                                         start=-CYCLE_START,
                                         end=-CYCLE_START+CS_DURATION+DELAY)
licks_tc_us = 1.*ut.compute_licks_during(licks, cycles,
                                         start=-CYCLE_START+CS_DURATION,
                                         end=-CYCLE_START+CS_DURATION+DELAY+AFTER_DELAY_DURATION)
licks_cs_tc_us = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START,
                                      end=-CYCLE_START+CS_DURATION+DELAY+AFTER_DELAY_DURATION)
lickrates_bs = 1.*licks_bs/(DELAY+AFTER_DELAY_DURATION)
lick_ratios = np.nan_to_num(1.*(licks_tc_us-licks_bs)/(licks_tc_us+licks_bs))
lick_di = np.nan_to_num(1.*(np.mean(licks_tc[is_rewardedt]-licks_bs[is_rewardedt]) -
                            np.mean(licks_tc[is_CSm_enabledt]-licks_bs[is_CSm_enabledt]))/np.sqrt(0.5*(np.std(licks_tc)**2+np.std(licks_bs)**2)))
    
good_lick_trials = (licks_bs+licks_tc_us) >= 5

is_errCSmt = (lick_ratios>0.8) * ((licks_tc_us+licks_bs) > 4) * is_CSm_enabledt
print is_errCSmt.sum()

is_corrCSmt = ((licks_tc_us)==0) * is_CSm_enabledt
print is_corrCSmt.sum()

0
3


/home/fabios/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in divide


In [49]:
# clean up artefact events at the beginning of each cycle - will need for odors, but not sucrose and shock as
# suc and shock was continuous imaging
#for s, e in cycles:
#    if s>np.max(time_ax): break
#    events[np.where(time_ax>=s)[0][0]] = 0

licks = np.r_[ut.parse_behavior(behavior, 'LICK')]
licks_bs = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START-DELAY,
                                      end=-CYCLE_START)  # w.r.t. cycle start
licks_cs = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START,
                                      end=-CYCLE_START+CS_DURATION)
licks_tc = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START+CS_DURATION,
                                         end=-CYCLE_START+CS_DURATION+DELAY)
licks_cs_tc = 1.*ut.compute_licks_during(licks, cycles,
                                         start=-CYCLE_START,
                                         end=-CYCLE_START+CS_DURATION+DELAY)
licks_tc_us = 1.*ut.compute_licks_during(licks, cycles,
                                         start=-CYCLE_START+CS_DURATION,
                                         end=-CYCLE_START+CS_DURATION+DELAY+AFTER_DELAY_DURATION)
licks_cs_tc_us = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START,
                                      end=-CYCLE_START+CS_DURATION+DELAY+AFTER_DELAY_DURATION)
lickrates_bs = 1.*licks_bs/(DELAY+AFTER_DELAY_DURATION)
lick_ratios = np.nan_to_num(1.*(licks_tc_us-licks_bs)/(licks_tc_us+licks_bs))
lick_di = np.nan_to_num(1.*(np.mean(licks_tc[is_rewardt]-licks_bs[is_rewardt]) -
                            np.mean(licks_tc[is_CSmt]-licks_bs[is_CSmt]))/np.sqrt(0.5*(np.std(licks_tc)**2+np.std(licks_bs)**2)))
    
good_lick_trials = (licks_bs+licks_tc_us) >= 5

is_errCSmt = (lick_ratios>0.8) * ((licks_tc_us+licks_bs) > 4) * is_CSmt
print is_errCSmt.sum()

is_corrCSmt = ((licks_tc_us)==0) * is_CSmt
print is_corrCSmt.sum()
print is_CSmt
print licks_tc_us

In [93]:
time_ax_single = ut.extract_single_cycle_time_ax(time_ax, cycles,
                                                 cycle_duration=CYCLE_DURATION, cycle_start=CYCLE_START)

## find time of first lick following reward delivery (ie, consumption of reward)

In [94]:
#establishes when the delivered sucrose was actually consumed by animal (first lick after delivery), wrt delivery time
consumption_times = []
for s, e in cycles[is_STIM5t]:
    try:
        r = rewards[(rewards>=s)*(rewards<e)][0]
        later_licks = licks-r
        consumption_times.append(later_licks[(later_licks>=0)][0])
    except:
        consumption_times.append(np.nan)

In [95]:
consumption_times = np.r_[consumption_times]
print consumption_times

[ 0.10690293  0.08463464  0.11087014  0.04751582  0.04157589  0.04475012
  0.11582082         nan  0.09899229         nan  0.04961338  0.01556618
  0.06421069  0.07226613  0.01167457  0.06533385  0.04961527  0.0128689
  0.04850601         nan  0.067131    0.07296792  0.06533849  0.02573942
  0.02237695  0.03210601  0.02474868  0.18116534  0.07820337  0.05642806
  0.16335059  0.09534464  0.07588688  0.03167985  0.09923908  0.06731873
  0.07588458  0.09206864  0.10702113  0.07722077]


In [96]:
#find trials where consumption took place at reasonable time following delivery
is_consumed = consumption_times<1.5

/home/fabios/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in less
  from ipykernel import kernelapp as app


In [97]:
print is_consumed
print sum(is_consumed)

[ True  True  True  True  True  True  True False  True False  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
37


In [98]:
i=0
is_consumed_STIM5t = []
for x,y in zip(is_STIM5t,range(len(is_STIM5t))):
    #print x,y
    if x == True:
        is_consumed_STIM5t.append(is_STIM5t[y]*is_consumed[i])
        i=i+1
    if x == False:
        is_consumed_STIM5t.append(False)
#print is_consumed_STIM5t
print sum(is_consumed_STIM5t)

37


#find the absolute times when consumption takes place
#this doesn't work if there are any reward trials where reward was not earned
consumptions = consumption_times + rewards

In [99]:
save_workspace(db)

Could not store variable 'ET'. Skipping...
Could not store variable 'di'. Skipping...
Could not store variable 'pickle'. Skipping...
Could not store variable 'os'. Skipping...
Could not store variable 'IPython'. Skipping...
Could not store variable 'pt'. Skipping...
Could not store variable 'sstats'. Skipping...
Could not store variable 'pl'. Skipping...
Could not store variable 'ut'. Skipping...
Could not store variable 'ip'. Skipping...
Could not store variable 'np'. Skipping...
Could not store variable 'sys'. Skipping...


### 